In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import joblib

In [23]:
df = pd.read_csv('../data/processed/engineered_inventory.csv')

In [24]:
df.columns

Index(['Item_ID', 'Item_Name', 'Category', 'Jan_Demand', 'Feb_Demand',
       'Mar_Demand', 'Apr_Demand', 'May_Demand', 'Jun_Demand', 'Jul_Demand',
       'Aug_Demand', 'Sep_Demand', 'Oct_Demand', 'Nov_Demand', 'Dec_Demand',
       'Total_Annual_Units', 'Price_Per_Unit', 'Total_Sales_Value',
       'Cumulative_Sales', 'Cumulative_Sales_Percentage', 'ABC_Class',
       'Demand_Mean', 'Demand_Std', 'Demand_CV', 'XYZ_Class'],
      dtype='object')

In [25]:
print(df[['ABC_Class', 'XYZ_Class']].value_counts())

ABC_Class  XYZ_Class
C          X            687
B          X            165
A          X             87
C          Y             54
B          Y              5
C          Z              2
Name: count, dtype: int64


In [26]:
X = df.drop(columns=['Item_ID', 'Item_Name','ABC_Class','XYZ_Class', 
                     'Cumulative_Sales', 'Cumulative_Sales_Percentage',], errors='ignore')

print('Feature Shape', X.shape)
print('Sample Features :',X.head())

Feature Shape (1000, 19)
Sample Features :   Category  Jan_Demand  Feb_Demand  Mar_Demand  Apr_Demand  May_Demand  \
0  Grocery        4991        4663        5068        4885        5127   
1  Grocery        4185        4087        3660        3774        4391   
2  Grocery        3079        3185        3081        2794        3313   
3  Grocery        5180        4626        5010        4802        4856   
4  Grocery        4871        4994        4740        4958        4428   

   Jun_Demand  Jul_Demand  Aug_Demand  Sep_Demand  Oct_Demand  Nov_Demand  \
0        4705        5130        5129        5127        4840        4774   
1        3906        3699        4155        3936        3646        3959   
2        3329        3168        3241        3309        2997        2781   
3        5361        5504        4910        4631        4959        4867   
4        4889        4754        4672        4477        4544        4553   

   Dec_Demand  Total_Annual_Units  Price_Per_Unit

In [27]:
y_abc = df['ABC_Class']
y_xyz = df['XYZ_Class']

In [28]:
print("ABC_Class distribution:\n", y_abc.value_counts())
print("\nXYZ_Class distribution:\n", y_xyz.value_counts())


ABC_Class distribution:
 ABC_Class
C    743
B    170
A     87
Name: count, dtype: int64

XYZ_Class distribution:
 XYZ_Class
X    939
Y     59
Z      2
Name: count, dtype: int64


# This is A Problem, We will use more advance StratKflod

In [37]:
labels = {
    "ABC" : y_abc,
    "XYZ" : y_xyz
}

result = {}

for label_name, target in labels.items(): 
    print(f'\n Training {label_name}_Classifier')

    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print('Classification Reports for {label_name}_Classifier : \n')
    print(classification_report(y_test, y_pred))

    print('Confusion Matrix : \n {confusion_matrix(y_test,y_pred)}')


 Training ABC_Classifier


ValueError: could not convert string to float: 'Grocery'

In [ ]:
from sklearn.preprocessing import LabelEncoder

X_encoded = X.copy()

for col in X_encoded.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col])


In [41]:
cleaned_df = X_encoded.copy()
cleaned_df['ABC_Class'] = y_abc
cleaned_df['XYZ_Class'] = y_xyz
cleaned_df.to_csv('../data/processed/cleaned_inventory.csv', index=False)


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import joblib

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

labels = {
    "ABC": y_abc,
    "XYZ": y_xyz
}

models = {}

for label_name, target in labels.items():
    print(f"\n🔧 Training {label_name}_Classifier")

    model = RandomForestClassifier(random_state=42)
    scores = cross_val_score(model, X, target, cv=skf, scoring='accuracy')

    print(f"✅ Mean Accuracy for {label_name}_Classifier: {scores.mean():.4f}")
    print(f"📊 Accuracy Scores across folds: {scores}")

    # Fit final model on full data
    model.fit(X, target)
    models[label_name] = model  # Save for later

    # Save to file
    joblib.dump(model, f"{label_name.lower()}_classifier.pkl")



🔧 Training ABC_Classifier
✅ Mean Accuracy for ABC_Classifier: 0.9960
📊 Accuracy Scores across folds: [1.    0.99  0.995 0.995 1.   ]


🔧 Training XYZ_Classifier


c:\Users\user\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


✅ Mean Accuracy for XYZ_Classifier: 0.9960
📊 Accuracy Scores across folds: [1.    1.    1.    0.995 0.985]



In [43]:
print(df['XYZ_Class'].value_counts())

XYZ_Class
X    939
Y     59
Z      2
Name: count, dtype: int64


In [44]:
import joblib
joblib.dump(abc_model, 'abc_classifier.pkl')
joblib.dump(xyz_model, 'xyz_classifier.pkl')


NameError: name 'abc_model' is not defined